# MF2 TESTING

In [ ]:
import sandy
import matplotlib.pyplot as plt

## LRU=0  (Thorium 233 in jeff 3.3)

In [ ]:
tape = sandy.get_endf6_file("jeff_33", "xs", 902330)

In [ ]:
dic = sandy.read_mf2(tape,tape.mat[0])
print(dic)

In [ ]:
tab = sandy.write_mf2(dic)
print(tab)

## LRU = 1 

### LRF = 1 or LRF = 2 (Thorium 230 in jeff 3.3)

In [ ]:
tape = sandy.get_endf6_file("jeff_33", "xs", 902300)

In [ ]:
dic = sandy.read_mf2(tape,tape.mat[0])

In [ ]:
tab = sandy.write_mf2(dic)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5),dpi = 100)
x = []
y = []
L = dic['NIS'][90230]['NER'][(1e-05, 251.0)]["L"]
for i in L.keys():
    for j in range(len(L[i]["RES_PAR"])):
        x.append(L[i]["RES_PAR"][j]["ER"])
        y.append(L[i]["RES_PAR"][j]["GT"])
ax = plt.stem(x, y, linefmt='blue', markerfmt='D')
plt.xlabel('Resonance energy (ev)')
plt.ylabel('Resonance total width')
plt.xlim(0)
plt.ylim(0)
plt.show()

### LRF = 3  (Uranium 238 in Jeff 3.3)

In [ ]:
tape = sandy.get_endf6_file("jeff_33", "xs", 922380)

In [ ]:
dic = sandy.read_mf2(tape,tape.mat[0])

In [ ]:
tab = sandy.write_mf2(dic)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5),dpi = 100)
L = dic["NIS"][92238]["NER"][(1e-05, 20000.0)]["L"]
for i in L.keys():
    x = []
    y = []
    for j in range(len(L[i]["RES_PAR"])):
        x.append(L[i]["RES_PAR"][j]["ER"])
        y.append(L[i]["RES_PAR"][j]["GN"])
    ax = plt.stem(x, y, label= f"l = {i}", linefmt='blue', markerfmt='D')
plt.xlabel('Resonance energy (ev)')
plt.ylabel('Radiation width')
plt.title('U238 Resonance parameters')

plt.xscale("log")
plt.yscale("log")
plt.legend()
plt.show()

### LRF = 7  (Iron 54 in ENDF/B-VIII.0)

In [ ]:
tape = sandy.get_endf6_file("endfb_80", "xs", 260540)

In [ ]:
dic = sandy.read_mf2(tape,tape.mat[0])

In [ ]:
dic['NIS'][26054]['NER'][(1e-05, 1036000.0)]['J'].keys()

In [ ]:
tab = sandy.write_mf2(dic)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5), dpi = 100)
x = []
y1 = []
y2 = []
J = dic['NIS'][26054]['NER'][(1e-05, 1036000.0)]['J'][(1.5, 0.0)]["RES_PAR"]
for k in range(len(J)):
    y1.append(J[k]["GAM"][0])
    y2.append(J[k]["GAM"][1])
    x.append(J[k]["ER"])
ax = plt.stem(x, y1, markerfmt='D', label = 'GAM 1')
ax = plt.stem(x, y2, markerfmt='o', label = 'GAM 2')
plt.xlabel('Resonance energy (ev)')
plt.ylabel('Channel width in eV')
plt.xscale("linear")
plt.yscale("log")
plt.legend()
plt.show()

## LRU = 2

### LFW = 0 and LRF = 1  (Iron 58 in ENDF/B-VIII.0)

In [ ]:
tape = sandy.get_endf6_file("endfb_80", "xs", 260580)

In [ ]:
dic = sandy.read_mf2(tape,tape.mat[0])

In [ ]:
tab = sandy.write_mf2(dic)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5), dpi = 100)
x = []
y = []
L = dic['NIS'][26058]['NER'][(350000.0, 3000000.0)]["L"]
for i in L.keys():
    for j in range(len(L[i]["RES_PAR"])):
        y.append(L[i]["RES_PAR"][j]["D"])
    for k in range(len(L[i]["RES_PAR"])):
        x.append(i)
    ax = plt.scatter(x, y , label = f"j = {L[i]['RES_PAR'][j]['AJ']}" )
plt.xlabel('Quantique number l')
plt.ylabel('Average level spacing for resonances with spin J')
plt.xlim(0,3)
plt.legend()
plt.show()

### LRF = 2  (Uranium 238 in Jeff 3.3)

In [ ]:
tape = sandy.get_endf6_file("jeff_33", "xs", 922350)

In [ ]:
dic = sandy.read_mf2(tape,tape.mat[0])

In [ ]:
tab = sandy.write_mf2(dic)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5),dpi = 100)
L = dic["NIS"][92235]["NER"][(2250.0, 46200.0)]["L"]
for l in L.keys():
    for j in L[l]["J"].keys():
        x = []
        y = []
        for k in range(len(L[l]["J"][j])):
            x.append(L[l]["J"][j]["RES_PAR"][k]["ES"])
            y.append(L[l]["J"][j]["RES_PAR"][k]["GG"])
        ax = plt.stem(x, y, linefmt='grey', markerfmt='D', label=f"{l,j}")
        plt.xlabel('Resonance energy (ev)')
        plt.ylabel('Average radiation width')
        plt.xlim(2200, 7100)
        plt.ylim(0.0376, 0.0388)
        plt.legend()